<a href="https://colab.research.google.com/github/nilanahar/MMAI_894_DeepLearning_Project_TeamRosedale/blob/main/MMAI_894_Team_Project_ResNet%2BGoogleWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up Environment

#### Packages to Load

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import json
import joblib
import os
import re
import ast
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# from wordcloud import WordCloud
# from collections import Counter

# import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

!pip install optuna
import optuna

In [ ]:
# define my google drive
drive.mount("/content/drive", force_remount=True)

#### Functions to Use (Must Run)

In [ ]:
# Function to load multiple JSON data
def load_json(file_path):
    if file_path: # Check if file_path is not None
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return None # or handle the None case differently, like returning an empty dictionary

def json_to_pdDf(mcq_file_path, oeq_file_path, answers_file_path):
    # Now you can use this function to load multiple JSON files
    mcq_json_data = load_json(mcq_file_path)
    oeq_json_data = load_json(oeq_file_path)

    mcq_df = pd.json_normalize(mcq_json_data['questions'])[['image_id', 'question_id', 'question', 'multiple_choices']]
    oeq_df = pd.json_normalize(oeq_json_data['questions'])[['image_id', 'question_id', 'question']]

    # Check if answers_file_path is None
    if answers_file_path is not None:
        answers_json_data = load_json(answers_file_path)
        answers_df = pd.json_normalize(answers_json_data['annotations'])[['image_id', 'question_id', 'question_type', 'answers', 'multiple_choice_answer', 'answer_type']]
    else:
        # Create an empty DataFrame if answers_file_path is None
        answers_df = pd.DataFrame(columns=['image_id', 'question_id', 'question_type', 'answers', 'multiple_choice_answer', 'answer_type'])

    return mcq_df, oeq_df, answers_df


def tabularize_answers_df(answers_df):
    # Create a unique identifier for each row to track back after transformations
    answers_df['index'] = answers_df.index
    # Explode the 'answers' column into separate rows, while keeping other columns intact
    answers_expanded_df = answers_df.explode('answers')
    # normalization as before
    answers_details = pd.json_normalize(answers_expanded_df['answers'])
    answers_details['index'] = answers_expanded_df.index
    # Concatenate back to the actual answers DataFrame
    answers_expanded_df = pd.merge(answers_df, answers_details, on='index', how='left')
    answers_merged_df = answers_expanded_df[['image_id', 'question_id', 'question_type',
                                              'multiple_choice_answer', 'answer_type',
                                              'answer', 'answer_id', 'answer_confidence',
                                              'index']]
    answers_merged_df = answers_merged_df.rename(columns={'multiple_choice_answer': 'target_answer'})

    return answers_merged_df

# Function to extract features from multiple JSON files
def extract_qna_features(mcq_file_path, oeq_file_path, answers_file_path):

    mcq_df, oeq_df, answers_df = json_to_pdDf(mcq_file_path, oeq_file_path, answers_file_path)
    answers_merged_df = tabularize_answers_df(answers_df)

    questions_merged_df = pd.merge(mcq_df, oeq_df, on=['image_id', 'question_id'], how='left', suffixes=('_oeq', '_mcq'))
    final_qna_df = pd.merge(questions_merged_df, answers_merged_df, on=['image_id', 'question_id'], how='left')

    return mcq_df, oeq_df, answers_merged_df, final_qna_df


# Function to extract last five digits from file names
def extract_last_five_digits(filename):
    match = re.search(r'(\d{5})\.png$', filename)
    if match:
        return match.group(1)
    else:
        raise ValueError(f"Filename {filename} does not match the expected pattern.")

def extract_image_features(model, images_file_path):
    images_features_dict = {}
    for img_name in tqdm(os.listdir(images_file_path)):
        img_path = os.path.join(images_file_path, img_name)

        # Extract image_id (adjust slice according to your filename structure)
        image_id = img_name[-9:-4]  # Example: extracting last 5 digits before file extension

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(224, 224))
        x = preprocess_input(np.expand_dims(image.img_to_array(img), axis=0))

        # Extract features and store them in the dictionary
        features = model.predict(x)
        images_features_dict[image_id] = features.flatten()

    images_features_df = pd.DataFrame.from_dict(images_features_dict, orient='index')
    images_features_df.reset_index(inplace=True)
    images_features_df.rename(columns={'index': 'image_id'}, inplace=True)

    return images_features_df


def complete_datasets(qna_df, images_features_df):
    qna_df = qna_df.sort_values(by='image_id').reset_index(drop=True)
    images_features_df = images_features_df.sort_values(by='image_id').reset_index(drop=True)
    images_features_df['image_id'] = images_features_df['image_id'].astype(int)
    # Merge the 3 dataframes into one (base dataset is image_df (unique), left join qna on image id)
    complete_training_set = pd.merge(images_features_df, qna_df, on='image_id', how='left')
    return complete_training_set

def preprocess_text(text):
    """
    Improved text preprocessing function that retains context and meaning.
    - Converts text to lowercase.
    - Removes non-alphabetic characters.
    - Tokenizes, removes only non-essential stopwords, and lemmatizes.
    """
    if pd.isnull(text):
        return ''

    # Convert text to lowercase
    text = text.lower()
    # Remove non-alphabetic characters (but keep question marks and spaces)
    text = re.sub(r'[^a-z\s\?]', '', text)
    # Tokenize text into words
    words = text.split()
    # Create a lemmatizer object
    lemmatizer = WordNetLemmatizer()
    # Define stop words
    stop_words = set(stopwords.words('english')) # Define stop_words here
    essential_stopwords = {"how", "how many", "what", "when", "why", "which", "where", "who"}
    # Remove stopwords, but retain essential question words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words or word in essential_stopwords]
    # Join words back into a single string
    return ' '.join(words)


def get_sentence_embedding(sentence, model):
    """
    Calculates the sentence embedding by averaging the word embeddings of all words in the sentence.
    If a word is not found in the model, it's embedding is considered as a zero vector.
    """
    # Convert series to string
    if isinstance(sentence, pd.Series):
        sentence = ' '.join(sentence.astype(str).tolist())
    words = sentence.split()
    # Removed .wv since model is already a KeyedVectors object
    word_vectors = [model[word] for word in words if word in model]

    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)

    return np.mean(word_vectors, axis=0)


# Modified filter_choices_by_cosine_similarity function using Word2Vec
def filter_choices_by_word2vec_similarity(row, model):
    """
    Filters multiple-choice options based on their cosine similarity with the question using Word2Vec embeddings.
    Args:
    - row: A DataFrame row containing a 'question' and 'multiple_choices' columns.
    - model: Trained Word2Vec model.

    Returns:
    - A list of filtered multiple-choice options based on cosine similarity.
    """
    question = row['question']
    choices = row['multiple_choices']  # Assume 'multiple_choices' column has a list of answer options

    # Step 1: Generate Word2Vec embeddings for the question
    question_embedding = get_sentence_embedding(question, model)

    # Step 2: Generate Word2Vec embeddings for each choice
    choice_embeddings = [get_sentence_embedding(choice, model) for choice in choices]

    # Step 3: Calculate cosine similarity between question embedding and each choice embedding
    similarity_scores = cosine_similarity([question_embedding], choice_embeddings)[0]

    # Step 4: Retain choices with a high similarity score (e.g., > 0.3)
    threshold = 0.3
    filtered_choices = [choices[i] for i, score in enumerate(similarity_scores) if score >= threshold]

    # Step 5: If no choices pass the threshold, keep original choices to avoid empty options
    return filtered_choices if filtered_choices else choices

# Step 1: Concatenate List into a Single Sentence for Each Row
def concatenate_list_to_sentence(row, list_column):
    """
    Concatenate all words in a list (in a single row) into a single string.
    Args:
    - row: A single row of the DataFrame.
    - list_column: Column name containing lists of words.

    Returns:
    - A single concatenated string of all words in the list.
    """
    return " ".join(row[list_column])  # Join words with a space separator


#### Define Constants

In [ ]:
### Load the pre-trained model that you wish to use to extract image features ###
# model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# wxord2vec_model = api.load('word2vec-google-news-300')
# joblib.dump(word2vec_model, "/content/drive/MyDrive/Colab Notebooks/894/Training/(2)_word2vec_model_for_2.2.joblib")
# model = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/(2)_word2vec_model_for_2.2.joblib")

# Step 1: Data Collection & Feature Extraction

#### Set Directories

In [ ]:
# Define the file path (adjust it to match your folder structure)
training_mcq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/Data/MultipleChoice_abstract_v002_train2015_questions.json"
training_oeq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/Data/OpenEnded_abstract_v002_train2015_questions.json"
training_answers_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/Data/abstract_v002_train2015_annotations.json"
training_images_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/Data/scene_img_abstract_v002_train2015"

validation_mcq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/Data/MultipleChoice_abstract_v002_val2015_questions.json"
validation_oeq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/Data/OpenEnded_abstract_v002_val2015_questions.json"
validation_answers_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/Data/abstract_v002_val2015_annotations.json"
validation_images_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/Data/scene_img_abstract_v002_val2015"

testing_mcq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Testing/Data/MultipleChoice_abstract_v002_test2015_questions.json"
testing_oeq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Testing/Data/OpenEnded_abstract_v002_test2015_questions.json"
testing_images_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Testing/Data/scene_img_abstract_v002_test2015"

### 1.1 Training Datasets

#### 1.1.1 Extracting Questions & Answers & Features

In [ ]:
train_mcq_df, train_oeq_df, train_answers_merged_df, train_final_qna_df = extract_qna_features(training_mcq_file_path, training_oeq_file_path, training_answers_file_path)
train_final_qna_df.head(5)

##### Save Workspace

In [ ]:
# Save the dataframes to Google Drive
joblib.dump(train_final_qna_df, "/content/drive/MyDrive/Colab Notebooks/894/Training/questions_answers_features.joblib")

#### 1.1.2 Extracting Images' Features


In [ ]:
train_images_features_df = extract_image_features(model, training_images_file_path)

##### Save Workspace

In [ ]:
joblib.dump(train_images_features_df, "/content/drive/MyDrive/Colab Notebooks/894/Training/images_features.joblib")

#### 1.1.3 Complete Training Dataset

##### Load Workspace

In [ ]:
train_qna_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/questions_answers_features.joblib")
train_images_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/images_features.joblib")
train_images_df.shape

In [ ]:
train_complete_df = complete_datasets(train_qna_df, train_images_df)
train_complete_df.head()

##### Save Workspace

In [ ]:
joblib.dump(train_complete_df, "/content/drive/MyDrive/Colab Notebooks/894/Training/complete_training_set_tabular.joblib")

### 1.2 Validation Datasets

#### 1.2.1 Extracting Questions & Answers


In [ ]:
valid_mcq_df, valid_oeq_df, valid_answers_merged_df, valid_final_qna_df = extract_qna_features(validation_mcq_file_path, validation_oeq_file_path, validation_answers_file_path)
valid_final_qna_df.head()

##### Save Workspace

In [ ]:
joblib.dump(valid_final_qna_df, "/content/drive/MyDrive/Colab Notebooks/894/Validation/questions_answers_features.joblib")

#### 1.2.2 Extracting Images' Features

In [ ]:
valid_images_features_df = extract_image_features(model, validation_images_file_path)

In [ ]:
valid_images_features_df.shape

##### Save Workspace

In [ ]:
joblib.dump(valid_images_features_df, "/content/drive/MyDrive/Colab Notebooks/894/Validation/images_features.joblib")

#### 1.2.3  Complete Validation Dataset

##### Load Workspace

In [ ]:
valid_qna_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/questions_answers_features.joblib")
valid_images_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/images_features.joblib")
valid_images_df.shape

In [ ]:
valid_complete_df = complete_datasets(valid_qna_df, valid_images_df)
valid_complete_df.head()

##### Save Workspace

In [ ]:
joblib.dump(valid_complete_df, "/content/drive/MyDrive/Colab Notebooks/894/Validation/complete_validation_set_tabular.joblib")

### 1.3 Testing Datasets

#### 1.3.1 Extracting Questions & Answers

In [ ]:
test_mcq_df, test_oeq_df, test_answers_df = json_to_pdDf(testing_mcq_file_path, testing_oeq_file_path, None)
test_final_qna_df = pd.merge(test_mcq_df, test_oeq_df, on=['image_id', 'question_id'], how='left', suffixes=('_oeq', '_mcq'))
test_final_qna_df.head()

##### Save Workspace

In [ ]:
joblib.dump(test_final_qna_df, "/content/drive/MyDrive/Colab Notebooks/894/Testing/questions_answers_features.joblib")

#### 1.3.2 Extracting Images' Features

In [ ]:
test_images_features_df = extract_image_features(model, testing_images_file_path)

##### Save Workspace

In [ ]:
joblib.dump(test_images_features_df, "/content/drive/MyDrive/Colab Notebooks/894/Testing/images_features.joblib")

#### 1.3.3 Complete Testing Dataset

##### Load Workspace

In [ ]:
test_qna_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Testing/questions_answers_features.joblib")
test_images_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Testing/images_features.joblib")
test_images_df.shape

In [ ]:
test_complete_df = complete_datasets(valid_qna_df, valid_images_df)
test_complete_df.head()

##### Save Workspace

In [ ]:
joblib.dump(test_complete_df, "/content/drive/MyDrive/Colab Notebooks/894/Testing/complete_testing_set_tabular.joblib")









<space>

# Step 2: Exploratory Data Analysis & Feature Engineering

##### Load Workspace

In [ ]:
df_train = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/complete_training_set_tabular.joblib")
df_valid = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/complete_validation_set_tabular.joblib")
df_train.head()

### 2.1 Visualization & Insights

##### Keeping only one question column since question_oeq = question_mcq

In [ ]:
# Check if two specific columns are identical in the DataFrame
are_questions_columns_equal = df_train['question_oeq'].equals(df_train['question_mcq'])  # Replace 'question_x' and 'question_y' with your actual column names

# Print the result
print("Are 'question_oeq' and 'question_mcq' identical? : ", "Yes" if are_questions_columns_equal else "No")

In [ ]:
df_train['question'] = df_train['question_oeq']
df_valid['question'] = df_valid['question_oeq']

df_train = df_train.drop(columns=['question_oeq', 'question_mcq'])
df_valid = df_valid.drop(columns=['question_oeq', 'question_mcq'])

##### Questions' Length Visualization

In [ ]:
# Analyze Question Length
df_train['question_length'] = df_train['question'].apply(lambda x: len(x.split()))
df_valid['question_length'] = df_valid['question'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 6))
sns.histplot(df_train['question_length'], kde=True, color='skyblue')
plt.title('Distribution of Question Length')
plt.xlabel('Number of Words in Question')
plt.ylabel('Frequency')
plt.show()

##### Questions' Common Words

In [ ]:
# # Most common words in question
# question_text = df_train['question']
# question_words = question_text.split()
# word_counts = Counter(question_words)
# common_words = word_counts.most_common(20)
# print("\nMost Common Words in Questions:")
# for word, count in common_words:
#     print(f"{word}: {count}")

##### Save Workspace

In [ ]:
joblib.dump(df_train, "/content/drive/MyDrive/Colab Notebooks/894/Training/(1)_dfTrain_unique_2.1.joblib")
joblib.dump(df_valid, "/content/drive/MyDrive/Colab Notebooks/894/Validation/(1)_dfValid_unique_2.1.joblib")

### 2.2 Feature Engineering & Embeddings/Encodings

##### Load Workspace

In [ ]:
df_train = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/(1)_dfTrain_unique_2.1.joblib")
df_valid = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/(1)_dfValid_unique_2.1.joblib")
df_train.head()

##### 2.2.1 Questions Text's Features' Engineering

In [ ]:
df_train['question_clean'] = df_train['question'].apply(preprocess_text)
df_valid['question_clean'] = df_valid['question'].apply(preprocess_text)
df_train[['question', 'question_clean']]

In [ ]:
# Apply the get_sentence_embedding function to each row of the 'question_clean' column
df_train['question_embedding'] = df_train['question_clean'].apply(lambda x: get_sentence_embedding(x, model))
df_valid['question_embedding'] = df_valid['question_clean'].apply(lambda x: get_sentence_embedding(x, model))
df_train.head()

##### 2.2.2 Answers Text's Features' Engineering

In [ ]:
df_train['answer_clean'] = df_train['answer'].apply(preprocess_text)
df_valid['answer_clean'] = df_valid['answer'].apply(preprocess_text)
df_train[['answer', 'answer_clean']]

In [ ]:
# Apply the get_sentence_embedding function to each row of the 'question_clean' column
df_train['answer_embedding'] = df_train['answer_clean'].apply(lambda x: get_sentence_embedding(x, model))
df_valid['answer_embedding'] = df_valid['answer_clean'].apply(lambda x: get_sentence_embedding(x, model))
df_train.head()

##### 2.2.3 Multiple Choices List's Features' Engineering

In [ ]:
df_train['filtered_multiple_choices'] = df_train.apply(lambda row: filter_choices_by_word2vec_similarity(row, model), axis=1)
df_valid['filtered_multiple_choices'] = df_valid.apply(lambda row: filter_choices_by_word2vec_similarity(row, model), axis=1)
df_train[['multiple_choices', 'filtered_multiple_choices']]

In [ ]:
df_train['filtered_multiple_choice_corpus'] = df_train.apply(lambda row: concatenate_list_to_sentence(row, 'filtered_multiple_choices'), axis=1)
df_valid['filtered_multiple_choice_corpus'] = df_valid.apply(lambda row: concatenate_list_to_sentence(row, 'filtered_multiple_choices'), axis=1)
df_train[['filtered_multiple_choices', 'filtered_multiple_choice_corpus']]

In [ ]:
# Apply the get_sentence_embedding function to each row of the 'question_clean' column
df_train['filtered_choices_embedding'] = df_train['filtered_multiple_choice_corpus'].apply(lambda x: get_sentence_embedding(x, model))
df_valid['filtered_choices_embedding'] = df_valid['filtered_multiple_choice_corpus'].apply(lambda x: get_sentence_embedding(x, model))
df_train.head()

##### 2.2.4 Encoding Selected Categorical Features

In [ ]:
# Initialize the LabelEncoder
label_encoder_answer_type = LabelEncoder()
label_encoder_confidence = LabelEncoder()

# Encode answer_type and answer_confidence separately
df_train['answer_type_encoded'] = label_encoder_answer_type.fit_transform(df_train['answer_type'])
df_valid['answer_type_encoded'] = label_encoder_answer_type.transform(df_valid['answer_type'])

df_train['answer_confidence_encoded'] = label_encoder_confidence.fit_transform(df_train['answer_confidence'])
df_valid['answer_confidence_encoded'] = label_encoder_confidence.transform(df_valid['answer_confidence'])

df_train

##### Save Workspace

In [ ]:
joblib.dump(df_train, "/content/drive/MyDrive/Colab Notebooks/894/Training/(2)_dfTrain_w2v_2.2.joblib")
joblib.dump(df_valid, "/content/drive/MyDrive/Colab Notebooks/894/Validation/(2)_dfValid_w2v_2.2.joblib")

# Step 3: Model Architecture & Training

##### Load Workspace

In [ ]:
df_train = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/(2)_dfTrain_w2v_2.2.joblib")
df_valid = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/(2)_dfValid_w2v_2.2.joblib")

df_train.head()

In [ ]:
# df_train.head().to_csv("/content/drive/MyDrive/Colab Notebooks/894/Training/(2)_dfTrain_2.2.csv", index=False)

### 3.1 Remove ID & Transformed Features

In [ ]:
id_features = ['index', 'image_id', 'question_id', 'answer_id', 'question_length']

categorical_features = [  'question', 'question_clean', 'question_type',
                          'multiple_choices',  'filtered_multiple_choices', 'filtered_multiple_choice_corpus',
                          'answer', 'answer_clean', 'answer_confidence', 'answer_type']

numerical_features = [col for col in df_train.columns if col not in categorical_features + id_features]

In [ ]:
df_train = df_train.drop(columns=id_features + categorical_features)
df_valid = df_valid.drop(columns=id_features + categorical_features)

##### Save Workspace

In [ ]:
joblib.dump(df_train, "/content/drive/MyDrive/Colab Notebooks/894/Training/(3)_dfTrain_3.1.joblib")
joblib.dump(df_valid, "/content/drive/MyDrive/Colab Notebooks/894/Validation/(3)_dfValid_3.1.joblib")

### 3.2 Reducing Dimensionality (Principal Component Analysis (PCA))

##### Load Workspace

In [3]:
df_train = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/(3)_dfTrain_3.1.joblib")
df_valid = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/(3)_dfValid_3.1.joblib")

# df_train = joblib.load("/Users/nila/Library/CloudStorage/GoogleDrive-nilaa.nahar@gmail.com/My Drive/Colab Notebooks/894/Training/(3)_dfTrain_3.1.joblib")
# df_valid = joblib.load("/Users/nila/Library/CloudStorage/GoogleDrive-nilaa.nahar@gmail.com/My Drive/Colab Notebooks/894/Validation/(3)_dfValid_3.1.joblib")

df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2044,2045,2046,2047,target_answer,question_embedding,answer_embedding,filtered_choices_embedding,answer_type_encoded,answer_confidence_encoded
0,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,0.383851,...,3.750608,4.098589,1.6235,0.379023,tan,"[0.065460205, 0.088765465, 0.0744222, 0.099446...","[0.007873535, 0.12890625, 0.02734375, 0.152343...","[0.0076060575, -0.006476122, 0.04698271, 0.114...",1,2
1,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,0.383851,...,3.750608,4.098589,1.6235,0.379023,tan,"[0.065460205, 0.088765465, 0.0744222, 0.099446...","[0.19140625, 0.05126953, -0.055419922, 0.06591...","[0.0076060575, -0.006476122, 0.04698271, 0.114...",1,2
2,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,0.383851,...,3.750608,4.098589,1.6235,0.379023,tan,"[0.065460205, 0.088765465, 0.0744222, 0.099446...","[0.059326172, 0.18457031, -0.029785156, 0.0336...","[0.0076060575, -0.006476122, 0.04698271, 0.114...",1,2
3,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,0.383851,...,3.750608,4.098589,1.6235,0.379023,tan,"[0.065460205, 0.088765465, 0.0744222, 0.099446...","[0.007873535, 0.12890625, 0.02734375, 0.152343...","[0.0076060575, -0.006476122, 0.04698271, 0.114...",1,2
4,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,0.383851,...,3.750608,4.098589,1.6235,0.379023,monkey bars,"[-0.017578125, 0.0061683655, 0.07950846, 0.121...","[-0.07119751, -0.01184082, -0.099731445, 0.209...","[0.0013631185, -0.073649086, -0.0022786458, 0....",1,2


##### Find out what is the optimal number of principal components you need for each feature

In [ ]:
from sklearn.decomposition import PCA

# Calculate explained variance for image input
pca_image_full = PCA().fit(df_train.iloc[:, :2048].values)
explained_variance_ratio_image = np.cumsum(pca_image_full.explained_variance_ratio_)

# Calculate explained variance for filtered choices embedding
pca_filtered_choices_full = PCA().fit(np.vstack(df_train['filtered_choices_embedding'].values))
explained_variance_ratio_filtered_choices = np.cumsum(pca_filtered_choices_full.explained_variance_ratio_)

# Plot cumulative explained variance for image input
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(explained_variance_ratio_image) + 1), explained_variance_ratio_image, label='Image Input')
plt.plot(np.arange(1, len(explained_variance_ratio_filtered_choices) + 1), explained_variance_ratio_filtered_choices, label='Filtered Choices Embedding')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs Number of PCA Components')
plt.axhline(y=0.90, color='r', linestyle='--', label='90% Variance Explained')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% Variance Explained')
plt.legend()
plt.grid(True)
plt.show()

# Determine the number of components for 90% and 95% explained variance
n_components_image_90 = np.argmax(explained_variance_ratio_image >= 0.90) + 1
n_components_image_95 = np.argmax(explained_variance_ratio_image >= 0.95) + 1

n_components_filtered_choices_90 = np.argmax(explained_variance_ratio_filtered_choices >= 0.90) + 1
n_components_filtered_choices_95 = np.argmax(explained_variance_ratio_filtered_choices >= 0.95) + 1

print(f"Number of components for 90% variance (Image Input): {n_components_image_90}")
print(f"Number of components for 95% variance (Image Input): {n_components_image_95}")
print(f"Number of components for 90% variance (Filtered Choices): {n_components_filtered_choices_90}")
print(f"Number of components for 95% variance (Filtered Choices): {n_components_filtered_choices_95}")


In [ ]:
# prompt: do the same pca above to questions and answers embedding

# Calculate explained variance for question embedding
pca_question_full = PCA().fit(np.vstack(df_train['question_embedding'].values))
explained_variance_ratio_question = np.cumsum(pca_question_full.explained_variance_ratio_)

# Calculate explained variance for answer embedding
pca_answer_full = PCA().fit(np.vstack(df_train['answer_embedding'].values))
explained_variance_ratio_answer = np.cumsum(pca_answer_full.explained_variance_ratio_)


# Plot cumulative explained variance for question and answer embeddings
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(explained_variance_ratio_question) + 1), explained_variance_ratio_question, label='Question Embedding')
plt.plot(np.arange(1, len(explained_variance_ratio_answer) + 1), explained_variance_ratio_answer, label='Answer Embedding')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs Number of PCA Components')
plt.axhline(y=0.90, color='r', linestyle='--', label='90% Variance Explained')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% Variance Explained')
plt.legend()
plt.grid(True)
plt.show()

# Determine the number of components for 90% and 95% explained variance
n_components_question_90 = np.argmax(explained_variance_ratio_question >= 0.90) + 1
n_components_question_95 = np.argmax(explained_variance_ratio_question >= 0.95) + 1

n_components_answer_90 = np.argmax(explained_variance_ratio_answer >= 0.90) + 1
n_components_answer_95 = np.argmax(explained_variance_ratio_answer >= 0.95) + 1


print(f"Number of components for 90% variance (Question Embedding): {n_components_question_90}")
print(f"Number of components for 95% variance (Question Embedding): {n_components_question_95}")
print(f"Number of components for 90% variance (Answer Embedding): {n_components_answer_90}")
print(f"Number of components for 95% variance (Answer Embedding): {n_components_answer_95}")


##### Use the optimal number of features (captures 95% variance) to compute your final PCs

In [4]:
from sklearn.decomposition import PCA

# Define the number of principal components for PCA
n_components_image = 452  # Reduce image input dimensions from 2048 to 452
n_components_filtered_choices = 131  # Reduce filtered_choices_embedding_input from 300 to 131
n_components_questions = 150  # Reduce question_embedding_input from 300 to 150
n_components_answers = 134  # Reduce answer_embedding_input from 300 to 134

# Apply PCA to image input
pca_image = PCA(n_components=n_components_image)
image_train_pca = pca_image.fit_transform(df_train.iloc[:, :2048].values)
image_valid_pca = pca_image.transform(df_valid.iloc[:, :2048].values)

# Apply PCA to question embedding
pca_questions = PCA(n_components=n_components_questions)
question_train_pca = pca_questions.fit_transform(np.vstack(df_train['question_embedding'].values))
question_valid_pca = pca_questions.transform(np.vstack(df_valid['question_embedding'].values))

# Apply PCA to answer embedding
pca_answers = PCA(n_components=n_components_answers)
answer_train_pca = pca_answers.fit_transform(np.vstack(df_train['answer_embedding'].values))
answer_valid_pca = pca_answers.transform(np.vstack(df_valid['answer_embedding'].values))

# Apply PCA to filtered choices embedding
pca_filtered_choices = PCA(n_components=n_components_filtered_choices)
filtered_choices_train_pca = pca_filtered_choices.fit_transform(np.vstack(df_train['filtered_choices_embedding'].values))
filtered_choices_valid_pca = pca_filtered_choices.transform(np.vstack(df_valid['filtered_choices_embedding'].values))

In [5]:
# Function to convert PCA result to DataFrame with preserved index
def pca_to_df(pca_result, df_original, prefix):
    """ Convert PCA results (NumPy array) to Pandas DataFrame and preserve the original DataFrame's index """
    df_pca = pd.DataFrame(pca_result, index=df_original.index,
                          columns=[f'{prefix}_pca_{i}' for i in range(pca_result.shape[1])])
    return df_pca

# Convert the PCA results to DataFrames with preserved row indices
df_image_train_pca = pca_to_df(image_train_pca, df_train, 'image')
df_image_valid_pca = pca_to_df(image_valid_pca, df_valid, 'image')
df_filtered_choices_train_pca = pca_to_df(filtered_choices_train_pca, df_train, 'filtered_choices')
df_filtered_choices_valid_pca = pca_to_df(filtered_choices_valid_pca, df_valid, 'filtered_choices')
df_questions_train_pca = pca_to_df(question_train_pca, df_train, 'questions')
df_questions_valid_pca = pca_to_df(question_valid_pca, df_valid, 'questions')
df_answers_train_pca = pca_to_df(answer_train_pca, df_train, 'answers')
df_answers_valid_pca = pca_to_df(answer_valid_pca, df_valid, 'answers')

In [6]:
df_train['target_answer'].head()

,target_answer
0,tan
1,tan
2,tan
3,tan
4,monkey bars


In [7]:
# Concatenate PCA results with the original `target_answer` column
df_train_pca = pd.concat([df_train[['target_answer', 'answer_type_encoded', 'answer_confidence_encoded']], df_image_train_pca, df_questions_train_pca, df_answers_train_pca, df_filtered_choices_train_pca], axis=1)
df_valid_pca = pd.concat([df_valid[['target_answer', 'answer_type_encoded', 'answer_confidence_encoded']], df_image_valid_pca, df_questions_valid_pca, df_answers_valid_pca, df_filtered_choices_valid_pca], axis=1)
df_valid_pca.head()

,target_answer,answer_type_encoded,answer_confidence_encoded,image_pca_0,image_pca_1,image_pca_2,image_pca_3,image_pca_4,image_pca_5,image_pca_6,...,filtered_choices_pca_121,filtered_choices_pca_122,filtered_choices_pca_123,filtered_choices_pca_124,filtered_choices_pca_125,filtered_choices_pca_126,filtered_choices_pca_127,filtered_choices_pca_128,filtered_choices_pca_129,filtered_choices_pca_130
0,no,2,2,-17.421425,-8.676399,1.046292,-7.151888,2.624744,-6.54362,-7.088371,...,0.029088,0.009248,-0.038447,0.018845,0.048154,-0.006325,0.005365,0.070331,-0.026675,-0.041262
1,no,2,2,-17.421425,-8.676399,1.046292,-7.151888,2.624744,-6.54362,-7.088371,...,0.029088,0.009248,-0.038447,0.018845,0.048154,-0.006325,0.005365,0.070331,-0.026675,-0.041262
2,no,2,2,-17.421425,-8.676399,1.046292,-7.151888,2.624744,-6.54362,-7.088371,...,0.029088,0.009248,-0.038447,0.018845,0.048154,-0.006325,0.005365,0.070331,-0.026675,-0.041262
3,yes,2,2,-17.421425,-8.676399,1.046292,-7.151888,2.624744,-6.54362,-7.088371,...,0.056799,-0.044716,-0.057458,0.005188,0.082895,-0.003959,-0.008489,0.039941,0.023345,-0.019131
4,no,2,2,-17.421425,-8.676399,1.046292,-7.151888,2.624744,-6.54362,-7.088371,...,0.029088,0.009248,-0.038447,0.018845,0.048154,-0.006325,0.005365,0.070331,-0.026675,-0.041262


##### Save Workspace

In [8]:
joblib.dump(df_train_pca, "/content/drive/MyDrive/Colab Notebooks/894/Training/(4)_dfTrain_3.2.joblib")
joblib.dump(df_valid_pca, "/content/drive/MyDrive/Colab Notebooks/894/Validation/(4)_dfValid_3.2.joblib")

['/content/drive/MyDrive/Colab Notebooks/894/Validation/(4)_dfValid_3.2.joblib']

In [18]:
df_train = df_train_pca
df_valid = df_valid_pca

In [19]:
# del pca_image, pca_questions, pca_answers, pca_filtered_choices
# del image_train_pca, image_valid_pca, question_train_pca, question_valid_pca, answer_train_pca, answer_valid_pca, filtered_choices_train_pca, filtered_choices_valid_pca
# del df_image_train_pca, df_image_valid_pca, df_questions_train_pca, df_questions_valid_pca, df_answers_train_pca, df_answers_valid_pca, df_filtered_choices_train_pca, df_filtered_choices_valid_pca
# del df_train_pca, df_valid_pca

### 3.3 Model Architecture & Training

##### Load Workspace

In [ ]:
df_train = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/(4)_dfTrain_3.2.joblib")
df_valid = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/(4)_dfValid_3.2.joblib")

df_train.head()

In [20]:
# Define the number of principal components for PCA
n_components_image = 452
n_components_filtered_choices = 131
n_components_questions = 150
n_components_answers = 134

##### Build the Model Architecture

In [24]:
# Define input layers for PCA-transformed inputs
question_embedding_input = Input(shape=(n_components_questions,), name='question_embedding_input')  # Use PCA components shape
answer_embedding_input = Input(shape=(n_components_answers,), name='answer_embedding_input')  # Use PCA components shape
filtered_choices_embedding_input = Input(shape=(n_components_filtered_choices,), name='filtered_choices_embedding_input')  # Use PCA components shape
image_input = Input(shape=(n_components_image,), name='image_input')  # Use PCA components shape

answer_type_input = Input(shape=(1,), name='answer_type_encoded')  # One-dimensional label-encoded feature
answer_confidence_input = Input(shape=(1,), name='answer_confidence_encoded')  # One-dimensional label-encoded feature

# Step 3: Concatenate all inputs
combined = Concatenate()([question_embedding_input, answer_embedding_input, image_input,
                          filtered_choices_embedding_input, answer_type_input, answer_confidence_input])

# Create dense layers and dropout layers for the combined inputs
x = Dense(64, activation='relu')(combined)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)

# One-hot encode the target labels for training and validation
train_labels = pd.get_dummies(df_train['target_answer']).values
valid_labels = pd.get_dummies(df_valid['target_answer']).values

# Check that the validation labels have the same number of classes as training labels
if valid_labels.shape[1] != train_labels.shape[1]:
    print(f"Mismatch in number of classes: Train Labels: {train_labels.shape[1]}, Validation Labels: {valid_labels.shape[1]}")
    # Create a new zero-filled array with the same number of columns as train_labels
    new_valid_labels = np.zeros((valid_labels.shape[0], train_labels.shape[1]))
    # Fill the new array with the original valid_labels values where classes exist
    new_valid_labels[:, :valid_labels.shape[1]] = valid_labels  # Copy existing classes
    print(f"Shape of adjusted `valid_labels`: {new_valid_labels.shape}")
    valid_labels = new_valid_labels

print(f"Final shape of `train_labels`: {train_labels.shape}")
print(f"Final shape of `valid_labels`: {valid_labels.shape}")

# Define the number of classes for output
num_classes = train_labels.shape[1]  # Get the correct number of classes from train_labels shape
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=[question_embedding_input, answer_embedding_input, image_input,
                      filtered_choices_embedding_input, answer_type_input, answer_confidence_input], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Mismatch in number of classes: Train Labels: 2521, Validation Labels: 1568
Shape of adjusted `valid_labels`: (300000, 2521)
Final shape of `train_labels`: (600000, 2521)
Final shape of `valid_labels`: (300000, 2521)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ question_embedding_input  │ (None, 150)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ answer_embedding_input    │ (None, 134)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ image_input (InputLayer)  │ (None, 452)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ filtered_choices_embeddi… │ (None, 131)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ answer_type_encoded       │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ answer_confidence_encoded │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 869)            │              0 │ question_embedding_in… │
│ (Concatenate)             │                        │                │ answer_embedding_inpu… │
│                           │                        │                │ image_input[0][0],     │
│                           │                        │                │ filtered_choices_embe… │
│                           │                        │                │ answer_type_encoded[0… │
│                           │                        │                │ answer_confidence_enc… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │         55,680 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 32)             │          2,080 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 32)             │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 2521)           │         83,193 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 140,953 (550.60 KB)

 Trainable params: 140,953 (550.60 KB)

 Non-trainable params: 0 (0.00 B)

None


##### Training the model

In [ ]:
# Step 3: Prepare PCA-Transformed Training Inputs
train_inputs = {
    'question_embedding_input': df_train.filter(like='questions_pca').values,  # Select columns with 'questions_pca' in the name
    'answer_embedding_input': df_train.filter(like='answers_pca').values,  # Select columns with 'answers_pca' in the name
    'filtered_choices_embedding_input': df_train.filter(like='filtered_choices_pca').values,  # Select columns with 'filtered_choices_pca' in the name
    'image_input': df_train.filter(like='image_pca').values,  # Select columns with 'image_pca' in the name
    'answer_type_encoded': np.reshape(df_train['answer_type_encoded'].values, (-1, 1)),  # Reshape to (num_samples, 1)
    'answer_confidence_encoded': np.reshape(df_train['answer_confidence_encoded'].values, (-1, 1))  # Reshape to (num_samples, 1)
}

# Apply the same PCA transformation to validation inputs using column filtering
valid_inputs = {
    'question_embedding_input': df_valid.filter(like='questions_pca').values,  # Select columns with 'questions_pca' in the name
    'answer_embedding_input': df_valid.filter(like='answers_pca').values,  # Select columns with 'answers_pca' in the name
    'filtered_choices_embedding_input': df_valid.filter(like='filtered_choices_pca').values,  # Select columns with 'filtered_choices_pca' in the name
    'image_input': df_valid.filter(like='image_pca').values,  # Select columns with 'image_pca' in the name
    'answer_type_encoded': np.reshape(df_valid['answer_type_encoded'].values, (-1, 1)),  # Reshape to (num_samples, 1)
    'answer_confidence_encoded': np.reshape(df_valid['answer_confidence_encoded'].values, (-1, 1))  # Reshape to (num_samples, 1)
}

# Step 4: Train the simplified model using PCA-transformed inputs
history = model.fit(train_inputs, train_labels, validation_data=(valid_inputs, valid_labels), epochs=10, batch_size=16)


Epoch 1/10
 9754/37500 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.6291 - loss: 1.7890

##### Saving the model

In [ ]:
# Save the trained model to a file
model.save('/content/drive/MyDrive/Colab Notebooks/894/Training/(5)_SimpleModel_3.4.h5')

In [ ]:
# Save the history to a text file
with open('/content/drive/MyDrive/Colab Notebooks/894/Training/(5)_SimpleModel_3.4_training_history.txt', 'w') as f:
    f.write("Training and Validation Performance:\n")
    for key, values in history.history.items():
        f.write(f"{key}: {values}\n")


# Step 4: Hyperparameter Tuning

##### Load Workspace


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/Colab Notebooks/894/Training/(5)_SimpleModel_3.4.h5')

df_train = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/(4)_dfTrain_3.2.joblib")
df_valid = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Validation/(4)_dfValid_3.2.joblib")

df_train.head()

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to tune
    n_units_layer1 = trial.suggest_int('n_units_layer1', 32, 64)  # Number of units in first Dense layer
    n_units_layer2 = trial.suggest_int('n_units_layer2', 16, 32)   # Number of units in second Dense layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)   # Dropout rate
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)  # Learning rate for Adam optimizer

    # Define the model architecture (similar to your previous one)
    question_embedding_input = Input(shape=(n_components_questions,), name='question_embedding_input')
    answer_embedding_input = Input(shape=(n_components_answers,), name='answer_embedding_input')
    filtered_choices_embedding_input = Input(shape=(n_components_filtered_choices,), name='filtered_choices_embedding_input')
    image_input = Input(shape=(n_components_image,), name='image_input')

    # Concatenate the PCA-transformed inputs
    combined = Concatenate()([question_embedding_input, answer_embedding_input, image_input, filtered_choices_embedding_input])

    # Fully connected layers with dynamic units and dropout
    x = Dense(n_units_layer1, activation='relu')(combined)
    x = Dropout(dropout_rate)(x)
    x = Dense(n_units_layer2, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    # Output layer (number of classes)
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=[question_embedding_input, answer_embedding_input, image_input,
                          filtered_choices_embedding_input], outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with early stopping to prevent overfitting
    model.fit(train_inputs, train_labels, validation_data=(valid_inputs, valid_labels),
              epochs=10, batch_size=16, verbose=0)

    # Evaluate the model on validation data
    val_loss, val_accuracy = model.evaluate(valid_inputs, valid_labels, verbose=0)

    # Return the validation accuracy as the objective value
    return val_accuracy

# Create the study object
study = optuna.create_study(direction='maximize')

# Start optimization
study.optimize(objective, n_trials=100)  # Number of trials can be adjusted